## TORCH.JIT.FORK

<b>
    
```python
torch.jit.fork(func, *args, **kwargs)
```
</b>

Creates an asynchronous task executing ```func``` and a reference to the value of the result of this execution. ```fork``` will return immediately, so the return value of <i>func</i> may not have been computed yet.

To force completion of the task and access the return value invoke ```torch.jit.wait``` on the Future. ```fork``` invoked with a ```func``` which returns a <i>```T```</i> is typed as ```torch.jit.Future[T]```.

### Parameters
- <b>func</b> :- A Python function or ```torch.nn.Module``` that will be invoked. If executed in TorchScript, it will execute asynchronously, otherwise it will not. Traced invocations of fork will be captured in the IR.
- **kwargs

### Returns

A reference to the execution of ```func```. The value <i>```T```</i> can only be accessed by forcing completion of ```func``` through ```torch.jit.wait```

<b>Return Type</b>

```torch.jit.Future[T]```

### Example (Fork a Free Function)

In [3]:
import torch
from torch import Tensor

def foo(a: Tensor, b: int) -> Tensor:
    return a + b

def bar(a):
    fut: torch.jit.Future[Tensor] = torch.jit.fork(foo, a, b=2)
    return torch.jit.wait(fut)

script_bar = torch.jit.script(bar)
input = torch.tensor(2)

# only the scripted version executes asynchronously
assert script_bar(input) == bar(input)

# trace is not run asynchronously, but fork is captured in IR
graph = torch.jit.trace(bar, (input,)).graph
assert "fork" in str(graph)

print("Script Bar: ", script_bar)
print(graph)

Script Bar:  <torch.jit.ScriptFunction object at 0x000001B7FDBD5130>
graph(%a : Long(requires_grad=0, device=cpu)):
  %6 : Future[Long(requires_grad=0, device=cpu)] = prim::fork_0(%a)
  %5 : Long(requires_grad=0, device=cpu) = aten::wait(%6)
  return (%5)
with prim::fork_0 = graph(%a : Long(requires_grad=0, device=cpu)):
  %1 : Long(requires_grad=0, device=cpu) = prim::Constant[value={2}]() # <ipython-input-3-1ea4c7954557>:5:0
  %2 : int = prim::Constant[value=1]() # <ipython-input-3-1ea4c7954557>:5:0
  %3 : Long(requires_grad=0, device=cpu) = aten::add(%a, %1, %2) # <ipython-input-3-1ea4c7954557>:5:0
  return (%3)



### Example (Fork a Module Method)

In [13]:
import torch
from torch import Tensor
class AddMod(torch.nn.Module):
    def forward(self, a: Tensor, b : int):
        return a + b
class Mod(torch.nn.Module):
    def __init__(self):
        super(Mod, self).__init__()
        self.mod = AddMod()
    def forward(self, input):
        fut = torch.jit.fork(self.mod, a, b=2)
        return torch.jit.wait(fut)
input = torch.tensor(2)
mod = Mod()
assert mod(input) == torch.jit.script(mod).forward(input)

NameError: name 'a' is not defined